# Tutorial 2: Association Mining

In the first tutorial we looked at how Python can be used to do basic data exploration. I hope it was useful to you-- Python is one of the most in-demand skills for data scientists. There are some good reasons for this. The first one is that relying only on software will restrict you to the bounds of that software. By being able to program, you are not as restricted, and can do things that require complex logic.

The second reason why a data scientist would want to learn Python is more theoretical. The Python programming language is designed to be highly portable, and as a result is one of the most versatile programming languages. Python hosts a large number of machine learning tools such as scikit-learn (machine learning), pandas (big data processing) and nltk (natural language processing), which do a lot of heavy lifting. Many of these features open new possibilities. We will explore one of them today by writing a simple association mining script using the same dataset as before.

## About the Aipriori Algorithm

The apriori association mining algorithm is a simple frequency item set mining algorithm, most commonly used to find associations over a transactional database. In the 1990s and early 2000s, this algorithm was used to discover a lot of data mining "gems" -- unexpected associations between purchases and behaviours. Today it is still commonly used by retailers and e-commerce companies to look at purchasing patterns. 

The algorithm looks at all of the transactions in a dataset and returns the most common associations between the items. It uses three measures: <i>support</i>, <i>confidence</i> and <i>lift</i>. Support describes the number of occurrences in the dataset. Confidence is how often the association is found to be true. Lift is the ratio of observed support to the independent observations of the items in the association. The wikipedia page gives a more detailed explanation: https://en.wikipedia.org/wiki/Association_rule_learning

### Installing the Apyori Library

Before we can get started, we will need to install the apyroi library. Yes, you read that right... aPYori. Python libraries often have letters "py" instead of "pi", just so you know that it is a PYthon library. If you haven't already, please type the following command in your Anaconda shell: 

<i>pip install apyori</i>

This will install the necessary library. After that, we can get started as we did before by loading the dataset. Once again, we will load the csv file content and save it into the list <i>ec</i>.

In [1]:
import csv #import the csv library for loading csv type files
from apyori import apriori #import the apriori library

ec = [] # we will declare a list to store the data

with open('data.csv', newline='') as csvfile: #this is an expression used by the csv library to open the file
    datareader = csv.reader(csvfile, delimiter=',', quotechar='|') #use the reader to read each row of the csv file and save as a list
    for row in datareader: #this is a loop for iterating through each row
        ec.append(row) #we will append the content of each row as row entry to the ec list

There are two main differences from last time. You will notice that we <i>imported</i> the apyori library at the top of the script. This is how we bring outside libraries into our Python environment. Also, unlike last time, we will use the regular reader, instead of a DictReader. The apyori library prefers lists, instead of dictionaries. Using the regular csv reader, we will get results that are lists of lists. The practical difference for us is that we will have to select list items by specifying their numbers, instead of their key like last time.

As before, it's usually a good practice to see what the string will print. If we run the following cell, we will get the output for the dataset that we loaded.

In [2]:
print(ec[4]) #prints the fifth item

['536365', '84029G', 'KNITTED UNION FLAG HOT WATER BOTTLE', '6', '12/1/2010 8:26', '3.39', '17850', 'United Kingdom']


Using the apyori library is simple. We simply call then apriori function on the dataset and it will create a bunch of associations for us. Let's do that, while saving the associations as the "associations" list. Let's print it to see what the associations look like.

In [3]:
associations = list(apriori(ec)) #use the apriori function to create associations

print(associations) #print the associations

[RelationRecord(items=frozenset({''}), support=0.2492664833643963, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({''}), confidence=0.2492664833643963, lift=1.0)]), RelationRecord(items=frozenset({'1'}), support=0.27377239763060285, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'1'}), confidence=0.27377239763060285, lift=1.0)]), RelationRecord(items=frozenset({'12'}), support=0.11268291782768357, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'12'}), confidence=0.11268291782768357, lift=1.0)]), RelationRecord(items=frozenset({'2'}), support=0.15101216069088963, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'2'}), confidence=0.15101216069088963, lift=1.0)]), RelationRecord(items=frozenset({'United Kingdom'}), support=0.9143178756620103, ordered_statistics=[OrderedStatistic(items_base=frozenset(), items_add=frozenset({'United Kingdom'}), confidence=

Wait! This is <u>not readable at all!</u>. By simply printing the list, we retrieved direct output from the associations. We can't use this on any sort of report! 

That said, we can sort of make out the details of the output. You can see the associations, support and confidence values. These are saved as lists within the list of associations. One way to manage this data is to only print the elements that we need, one by one. Let's loop through the results and only print the relevant data.

In [4]:
for a in associations: #loop through each association directly
    #corresponds to the list values for association, the support and the confidence.
    print("Association: " + str(list(a[0])) + ", Support: " + str(a[1]) + ", Confidence: " + str(a[2][0][2]))

Association: [''], Support: 0.2492664833643963, Confidence: 0.2492664833643963
Association: ['1'], Support: 0.27377239763060285, Confidence: 0.27377239763060285
Association: ['12'], Support: 0.11268291782768357, Confidence: 0.11268291782768357
Association: ['2'], Support: 0.15101216069088963, Confidence: 0.15101216069088963
Association: ['United Kingdom'], Support: 0.9143178756620103, Confidence: 0.9143178756620103
Association: ['', '1'], Support: 0.1383181709139894, Confidence: 0.5549007995262066
Association: ['', 'United Kingdom'], Support: 0.24653540255762027, Confidence: 0.9890435297601421
Association: ['1', 'United Kingdom'], Support: 0.2699267406026831, Confidence: 0.9859530870854679
Association: ['2', 'United Kingdom'], Support: 0.14481925042903804, Confidence: 0.9589906519215495
Association: ['', '1', 'United Kingdom'], Support: 0.13803768153383403, Confidence: 0.9979721436576124


That's better. However, it also revealed another problem. We only have 10 association rules.

This was caused by the apriori functions default values. By default, it only collects support above 0.1. This is far too low for our dataset, as there is a lot of variance in the purchases. We can override the default by stating it in the function. Let's try lowering it to 0.002 ... patterns that happen at least 1000 times in our dataset (<u>warning: this may take about a minute to process on some computers</u>).

In [5]:
associations = list(apriori(ec, min_support=0.002)) #min support changed to 0.002
print(len(associations))

765


We can once again look through these associations by looping and printing. Let's do that.

In [6]:
for a in associations:
    print("Association: " + str(list(a[0])) + ", Support: " + str(a[1]) + ", Confidence: " + str(a[2][0][2]))

Association: [''], Support: 0.2492664833643963, Confidence: 0.2492664833643963
Association: ['-1'], Support: 0.007720839253750623, Confidence: 0.007720839253750623
Association: ['-2'], Support: 0.0025742281928733553, Confidence: 0.0025742281928733553
Association: ['0'], Support: 0.0046409920466498125, Confidence: 0.0046409920466498125
Association: ['0.19'], Support: 0.0035153438762894206, Confidence: 0.0035153438762894206
Association: ['0.21'], Support: 0.002147958148031961, Confidence: 0.002147958148031961
Association: ['0.29'], Support: 0.008167407872155892, Confidence: 0.008167407872155892
Association: ['0.39'], Support: 0.015214703548559723, Confidence: 0.015214703548559723
Association: ['0.42'], Support: 0.04527135502205163, Confidence: 0.04527135502205163
Association: ['0.55'], Support: 0.014137033824804857, Confidence: 0.014137033824804857
Association: ['0.65'], Support: 0.01450609879869351, Confidence: 0.01450609879869351
Association: ['0.72'], Support: 0.002278976213762433, Co

Better still. If you scroll down, we see some interesting patterns start to emerge, such as:
<ul>
    <li>'United Kingdom', 'ASSORTED COLOUR BIRD ORNAMENT'</li>
    <li>'United Kingdom', 'WHITE HANGING HEART T-LIGHT HOLDER'</li>
    <li>'United Kingdom', '4.95', '47566', 'PARTY BUNTING'</li>
</ul>
Though interesting that some products are purchased from the UK at a certain price, it is clear that there is something missing. Typically, basket analysis is conducted <i>on baskets</i>. We seem to have output that is improperly structured for this task.

One option is to restructure the data. Fortunately, we are using a programming language, so this step is relatively simple. We can create a simple script that creates a new list of baskets for association mining. The script below does this and finishes by reporting the number of basket associations produced.

In [7]:
i = 1
basket = [] #specify a list for basket items
baskets = [] #a list of baskets
invoice = ec[1][0] #a placeholder for the previous invoice number

while i < len(ec): #iterate through each item sequentially
    if ec[i][0] != invoice: #if this is a new invoice
        baskets.append(basket) #append the baskets
        basket = [] #start a new basket
    basket.append(ec[i][2]) #add this item to basket
    invoice = ec[i][0] #specify this current invoice as the new previous invoice
    i += 1 #iterate to the next item

len(baskets)

25900

As before, we can use the apriori function to create the associations and the print statement to print the results. Let's only print results that also have a confidence of greater than 50%, meaning that at least half of the time, these things are purchased together.

In [8]:
associations = list(apriori(baskets, min_support=0.02))

for a in associations:
    if a[2][0][2] > 0.5:
        print("Association: " + str(list(a[0])) + ", Support: " + str(a[1]) + ", Confidence: " + str(a[2][0][2]))

Association: ['ALARM CLOCK BAKELIKE GREEN', 'ALARM CLOCK BAKELIKE RED '], Support: 0.024903474903474904, Confidence: 0.645
Association: ['RED RETROSPOT CHARLOTTE BAG', 'CHARLOTTE BAG PINK POLKADOT'], Support: 0.020308880308880308, Confidence: 0.6921052631578947
Association: ['SPACEBOY LUNCH BOX ', 'DOLLY GIRL LUNCH BOX'], Support: 0.020772200772200773, Confidence: 0.6255813953488373
Association: ['GARDENERS KNEELING PAD KEEP CALM ', 'GARDENERS KNEELING PAD CUP OF TEA '], Support: 0.021196911196911198, Confidence: 0.7176470588235294
Association: ['GREEN REGENCY TEACUP AND SAUCER', 'PINK REGENCY TEACUP AND SAUCER'], Support: 0.024864864864864864, Confidence: 0.609271523178808
Association: ['ROSES REGENCY TEACUP AND SAUCER ', 'GREEN REGENCY TEACUP AND SAUCER'], Support: 0.03027027027027027, Confidence: 0.7417218543046358
Association: ['JUMBO BAG RED RETROSPOT', 'JUMBO  BAG BAROQUE BLACK WHITE'], Support: 0.022895752895752897, Confidence: 0.6261879619852165
Association: ['JUMBO BAG RED RET

This is much more powerful information. We can see that, for instance, that 'ROSES REGENCY TEACUP AND SAUCER ', 'GREEN REGENCY TEACUP AND SAUCER', 'PINK REGENCY TEACUP AND SAUCER' are <i>purchased together 85% of the time</i>! From the perspective of marketing, we could use this information to learn more about those customers and why they are purchasing so many saucers, and what are their motivations for doing so at that point in time.

What other associations are out there? Feel free to play with the variables in this code to find new results!

## Challenge Question
What happens when you change the confidence to 0.2? Do you think these less frequent associations are interesting to marketers?